# Imports

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 255)
from bs4 import BeautifulSoup
import nltk
from nltk.stem.porter import *
from nltk.tokenize import word_tokenize
from nltk.tokenize.treebank import TreebankWordDetokenizer
from pathlib import Path
nltk.download('punkt')
from dotenv import load_dotenv
import os

In [ ]:
%cd "c:\\Users\\t1nipun\\Desktop\\BERDI\\esa-data-bank_banque-donnees-ees"
ROOT_PATH = os.getcwd()
from berdi.Database_Connection_Files.connect_to_sqlserver_database import connect_to_db

In [ ]:
nltk.download('stopwords', quiet=True)

`esa_csvs_raw.txt` is a tab-separated file of the raw data from the MySQL database. You will need to create this file if you want to include new data. The file contains the name of the CSV file as well as the contents of the CSV file (of the table). You need to make it tab-separated because MSSQL will not separate things properly if you use a comma because the CSV contents include commas. If you *really* want to use a comma, there is a way to do it where there will be quotes added to separate the fields, but you will need to look up how to do it.

In [ ]:
load_dotenv(
    dotenv_path= str(ROOT_PATH) + "\\berdi\\Database_Connection_Files\\.env", override=True
)
conn = connect_to_db()

df = pd.read_sql('''SELECT csvFileName, csvText FROM [DS_TEST].[BERDI].csvs;''', conn)
df.to_csv(str(ROOT_PATH) + "\\data\\processed\\csvs\\esa_csvs_raw.txt", index = False, sep = '\t', encoding = 'utf-8-sig')

In [ ]:
df_raw = pd.read_csv(str(ROOT_PATH) + "\\data\\processed\\csvs\\esa_csvs_raw.txt", sep='\t')

In [ ]:
df_raw.iloc[0]

In [ ]:
df_raw['csvText'] = df_raw['csvText'].str.slice(0, 30_000) # limit to 30k chars since Excel can't handle more than 32k

In [ ]:
assert (df_raw['csvText'].str.len()).max() <= 30_000 # sanity check

In [ ]:
df_index = pd.read_csv(str(ROOT_PATH) + "\\data\\interim\\Intermediate_Index_Files\\Table_Fig_Alignment_Index.csv", encoding='utf-8')

In [ ]:
df_index.head(2)

In [ ]:
df_merged = df_raw[df_raw['csvFileName'].isin(list(df_index['csvFileName']))].reset_index(drop=True)

In [ ]:
df_joined = pd.merge(df_index, df_merged, how='left', on='csvFileName')

In [ ]:
df_joined.head(2)

In [ ]:
len(df_joined) # 502

In [ ]:
df_joined.columns

In [ ]:
del df_raw
del df_index
del df_merged

## Adding New Projects to the Index File

In [ ]:
df_joined.rename(columns={"csvText": "text"}, inplace=True)
df_joined['label'] = -1

In [ ]:
df_joined["text"] = df_joined["Title"] + " " + df_joined["text"]

In [135]:
len(df_joined)

502

In [ ]:
df_joined['text'] = df_joined['text'].astype(str).apply(lambda x: BeautifulSoup(x, "html.parser").get_text(separator=' ')) # remove html tags
df_joined['text'] = df_joined['text'].replace('[^a-zA-Z0-9 ]', ' ', regex=True) # remove all non-alpha-numeric characters
df_joined['text'] = df_joined['text'].replace('\w{25,}', ' ', regex=True)
df_joined['text'] = df_joined['text'].replace('cid\d+', ' ', regex=True)
df_joined['text'] = df_joined['text'].replace(' s ', ' ', regex=True)
df_joined['text'] = df_joined['text'].replace(' +', ' ', regex=True) # remove all extra spaces in text

In [ ]:
df_joined.text[0]

In [ ]:
df_joined.to_csv(str(ROOT_PATH) + '\\data\\interim\\Intermediate_Index_Files\\esa_index_with_table_text_no_labels.csv', index=False, encoding='utf-8-sig')

In [ ]:
from sklearn.model_selection import train_test_split
train_df, df_for_labeling = train_test_split(df_joined, test_size=1000, random_state=42, shuffle=True)

In [ ]:
print(len(train_df))
print(len(df_for_labeling))

In [ ]:
train_df.head(2)

In [ ]:
df_for_labeling.head(2)

In [ ]:
df_for_labeling.to_csv(str(ROOT_PATH / 'data/interim/Intermediate_Index_Files/esa_index_for_labeling.csv'), index=False, encoding='utf-8')